In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
import os
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
from requests import get
from bs4 import BeautifulSoup
from pandas.util import hash_pandas_object

In [ ]:
data = pd.read_csv('data/en.openfoodfacts.org.products.csv', sep = '\t')

In [30]:
header = data.columns
odf = pd.DataFrame()
for index in header:
    odf[index] =( [index , sum(data[index].notnull())/1023451*100  , data[index].value_counts().head(4).index.tolist() ])
odf=odf.T
odf =odf.drop(columns=0)
odf=odf.rename(columns={1: "%", 2: "exemple top-4"})

In [ ]:
odfcut=odf[odf["%"]>=3]

In [ ]:
odfcut.to_csv('odfcut.csv')


Categories each product according to the PNNS

In [31]:
data.pnns_groups_1.value_counts()

unknown                    705708
Sugary snacks               56989
Milk and dairy products     42729
Fish Meat Eggs              41703
Cereals and potatoes        33594
Beverages                   28901
Fat and sauces              27723
Composite foods             25014
Fruits and vegetables       24014
Salty snacks                19645
sugary-snacks                3450
fruits-and-vegetables        2862
cereals-and-potatoes           43
salty-snacks                    5
Name: pnns_groups_1, dtype: int64

In [32]:
pnns1_category = {'unknown' : 'Unknown',
 'Sugary snacks' : 'Snacks', 
 'Milk and dairy products' : 'Dairies',
 'Composite foods' : 'Composite', 
 'Cereals and potatoes' : 'Starchy', 
 'Fish Meat Eggs' : 'Fish Meat Eggs',
 'Beverages' : 'Beverages',
 'Fat and sauces' : 'Fat Sauces',
 'Fruits and vegetables' : 'Fruits Vegetables',
 'Salty snacks' : 'Snacks',
 'fruits-and-vegetables' : 'Fruits Vegetables',
 'sugary-snacks' : 'Snacks',
 'cereals-and-potatoes' : 'Starchy',
 'salty-snacks' : 'Snacks'
}
data['Category_Food'] = data['pnns_groups_1'].replace(pnns1_category)

In [33]:
data.Category_Food.value_counts()

Unknown              705708
Snacks                80089
Dairies               42729
Fish Meat Eggs        41703
Starchy               33637
Beverages             28901
Fat Sauces            27723
Fruits Vegetables     26876
Composite             25014
Name: Category_Food, dtype: int64

More category from 'main_category_en' source

In [34]:
data['main_category_en'].value_counts().head(30)

Plant-based foods and beverages    85178
Snacks                             50961
Beverages                          42838
Dairies                            37304
Meats                              31040
Groceries                          23197
Meals                              17231
Spreads                             7299
Desserts                            7213
Seafood                             6947
Fruit juices                        6359
Canned foods                        4671
Frozen foods                        4107
Biscuits and cakes                  3283
Sweeteners                          2442
Dietary supplements                 1937
Vinegars                            1739
Sandwiches                          1737
Breakfasts                          1689
Fats                                1675
Farming products                    1603
Fish and meat and eggs              1602
Baby foods                          1390
Fruit-juices-and-nectars            1241
Pies            

In [35]:
maincategoryen_category = {'unknown' : 'Unknown',
'Plant-based foods and beverages' : 'Brevages',
'Snacks' : 'Snacks',
'Beverages' : 'Beverages',
'Dairies' : 'Dairies' ,
'Meats' : 'Fish Meat Eggs',
'Meals' : 'Composite',
'Desserts' : 'Snacks',
'Seafood' : 'Fish Meat Eggs',
'Fruit juices' : 'Brevages',
'Biscuits and cakes' : 'Snacks',
'Fats' : 'Fat Sauces', 
'Fish and meat and eggs' : 'Fish Meat Eggs',
'Fruit-juices-and-nectars' : 'Brevages'
                          }


data['Category Food bis'] = data['main_category_en'].replace(maincategoryen_category)

In [36]:
data['Category Food bis'].value_counts().head(20)

Brevages               92778
Snacks                 61457
Beverages              42838
Fish Meat Eggs         39589
Dairies                37304
Groceries              23197
Composite              17231
Spreads                 7299
Canned foods            4671
Frozen foods            4107
Sweeteners              2442
Dietary supplements     1937
Vinegars                1739
Sandwiches              1737
Breakfasts              1689
Fat Sauces              1675
Farming products        1603
Baby foods              1390
Pies                     974
Pickles                  923
Name: Category Food bis, dtype: int64

In [37]:
data[['Category Food bis','Category Food']]

,Category Food bis,Category Food
0,NaN,Unknown
1,NaN,Unknown
2,fr:xsf,Unknown
3,NaN,Unknown
4,NaN,Unknown
...,...,...
1023446,fr:tests,Unknown
1023447,NaN,Unknown
1023448,Snacks,Unknown
1023449,Fish Meat Eggs,Fish Meat Eggs
